# 简单图网络后门检测任务 PoC

In [1]:
import torch
torch.__version__

'1.10.1+cu102'

In [2]:
from dgl.data.dgl_dataset import DGLBuiltinDataset
from model_lib.mnist_cnn_model import Model0 as Model
import os 
import re
import torch
from torch import nn
import dgl

class HomoStrucBackdoorDataset(DGLBuiltinDataset):
    def __init__(self, mode='train', raw_dir='/home/ubuntu/date/hdd4/shadow_model_ckpt/mnist/models/',
                 force_reload=False, verbose=False, transform=None):
        mode = mode.lower()
        assert mode in ['train', 'valid', 'test'], "Mode not valid."
        self.mode = mode    
        self.x = []
        self.y = []
        _url = None
        super(HomoStrucBackdoorDataset, self).__init__(name='HomoBackdoorDT',
                                           raw_dir=raw_dir,
                                           force_reload=force_reload,
                                           verbose=verbose,
                                           url=_url,
                                           transform=transform)
    def process(self):
        pass
    
    def has_cache(self):
        pass
    
    def load(self):
        '''load dataset info'''
        
        for filename in os.listdir(self.raw_dir):
            if '.model' not in filename:
                # not a model
                continue
            idx_pattern = '[0-9]+'
            idx = re.findall(idx_pattern, filename)
            if self.mode == 'train':
                if int(idx[0]) < 2048 and 'target' not in filename:
                    # is a training model
                    self.x.append(filename)
                else:
                    continue
                # print(filename)
            elif self.mode == 'valid':
                if int(idx[0]) >= 2048 and 'target' not in filename:
                    self.x.append(filename)
                else:
                    continue
            else:
                # self.mode == 'test'
                if 'target' in filename:
                    self.x.append(filename)
                else:
                    continue
            # add co
            if 'benign' in filename:
                self.y.append(0)
            else:
                self.y.append(1)
        
    def __getitem__(self, idx):
        assert idx < len(self.x), "Out of index when get item."
        # load data, process and return
        g, y = self.load_g(idx)
        return g, y
        
    def __len__(self):
        return len(self.x)
    
    def get_x_y(self):
        return self.x, self.y
    
    def iter_y(self):
        for y in self.y:
            yield y
    
    def load_g(self, idx):
        x = os.path.join(self.raw_dir, self.x[idx])
        y = self.y[idx]
    #         print(label)
        CUDA_LAUNCH_BLOCKING=1
        basic_model = Model().cuda()
        t = torch.load(x)
        t = basic_model.load_state_dict(t)
        
        g = None
        with torch.no_grad():
            # nodes_feat 512 * 513
            nodes_feat = []
            cnt = 0
            # get conv1 nodes 
            conv1 = {}
            for weight in basic_model.conv1.weight:
                pad = nn.ZeroPad2d(padding=(254,254,253,254))
                feat = pad(weight[0])
                conv1[cnt] = feat
                nodes_feat.append(feat)
                cnt += 1

            # get conv2 nodes
            conv2 = {}
            for weight in basic_model.conv2.weight:
                pad = nn.ZeroPad2d(padding=(254,254,253,254))
                feat = pad(weight[0])
                conv2[cnt] = feat
                nodes_feat.append(feat)
                cnt += 1

            # get conv1 -> conv2 edges
            conv1_2 = []
            for src in conv1.keys():
                for dst in conv2.keys():
                    conv1_2.append([src, dst])


            # get fc node
            fc_index = cnt
            cnt += 1
            fc_node = torch.concat([basic_model.fc.weight, basic_model.fc.bias.reshape(512, 1)], 1)
            nodes_feat.append(fc_node)
            # print(fc_node.shape)

            # get conv2 -> fc edges
            conv2_fc = []
            for src in conv2.keys():
                conv2_fc.append([src, fc_index])

            # get output node
            out_index = cnt
            cnt += 1 
            out = torch.concat([basic_model.output.weight, basic_model.output.bias.reshape(10, 1)], 1)
            pad = nn.ZeroPad2d(padding=(0,0,251,251))
            out_node = pad(out)
            nodes_feat.append(out_node)

            # print(out_node.shape)

            # get fc -> output edge
            fc_out_edge = [[fc_index, out_index]]

            # get all nodes
            nodes_feat = torch.stack(nodes_feat)
            # print(nodes_feat.shape)
            # get all edges
            all_edges = torch.tensor(conv1_2 + conv2_fc + fc_out_edge).t().tolist()
            u, v = all_edges[0], all_edges[1]


            g = dgl.graph((u,v)).to('cuda')
            g.ndata['x'] = nodes_feat
        return g, y
dataset = HomoStrucBackdoorDataset(mode='test')
dataset.load()
print(len(dataset))

for x,y in dataset:
    print(x, y)
    break

768
Graph(num_nodes=50, num_edges=545,
      ndata_schemes={'x': Scheme(shape=(512, 513), dtype=torch.float32)}
      edata_schemes={}) 1


In [3]:
x, y = dataset.get_x_y()
for i in x:
    print(i)

target_trojB_113.model
target_trojM_21.model
target_benign_148.model
target_trojB_26.model
target_trojM_85.model
target_benign_10.model
target_trojM_122.model
target_trojB_187.model
target_benign_237.model
target_benign_233.model
target_trojM_159.model
target_trojB_17.model
target_trojM_91.model
target_trojB_38.model
target_trojM_117.model
target_benign_99.model
target_trojB_157.model
target_trojM_145.model
target_benign_67.model
target_benign_4.model
target_trojB_0.model
target_trojM_71.model
target_benign_107.model
target_trojM_33.model
target_trojM_115.model
target_trojM_60.model
target_benign_48.model
target_trojM_231.model
target_benign_220.model
target_trojB_130.model
target_benign_141.model
target_trojB_110.model
target_benign_234.model
target_benign_215.model
target_trojB_57.model
target_trojM_116.model
target_trojB_234.model
target_trojM_79.model
target_benign_124.model
target_trojB_22.model
target_benign_240.model
target_trojB_186.model
target_trojM_186.model
target_trojM_17.

In [4]:
def is_correct_labeled(x, y):
    cnt = 0
    error = 0
    for i,j in zip(x,y):
        if 'benign' in i and j == 0:
            cnt += 1
        elif 'benign' not in i and j == 1:
            cnt += 1
        else:
            error += 1
    if cnt != len(x) or error > 0:
        return False
    else:
        return True


In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import StratifiedKFold
from dgl.data import GINDataset
from dgl.dataloading import GraphDataLoader
from dgl.nn.pytorch.conv import GINConv
from dgl.nn.pytorch.glob import SumPooling
import argparse
from tqdm import tqdm

class MLP(nn.Module):
    """Construct two-layer MLP-type aggreator for GIN model"""
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.linears = nn.ModuleList()
        # two-layer MLP    
        self.linears.append(nn.Linear(input_dim, hidden_dim, bias=False))
        self.linears.append(nn.Linear(hidden_dim, output_dim, bias=False))
        self.batch_norm = nn.BatchNorm1d((hidden_dim))

    def forward(self, x):
        h = x
        h = F.relu(self.batch_norm(self.linears[0](h)))
        return self.linears[1](h)
    
class GIN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, pooling='sum'):
        super().__init__()
        self.ginlayers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        assert pooling in ['sum', 'avg', 'max']
        num_layers = 5
        # five-layer GCN with two-layer MLP aggregator and sum-neighbor-pooling scheme
        for layer in range(num_layers - 1): # excluding the input layer
            if layer == 0:
                mlp = MLP(input_dim, hidden_dim, hidden_dim)
            else:
                mlp = MLP(hidden_dim, hidden_dim, hidden_dim)
            self.ginlayers.append(GINConv(mlp, learn_eps=False)) # set to True if learning epsilon
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))
        # linear functions for graph sum poolings of output of each layer
        self.linear_prediction = nn.ModuleList()
        for layer in range(num_layers):
            if layer == 0:
                self.linear_prediction.append(nn.Linear(input_dim, output_dim))
            else:
                self.linear_prediction.append(nn.Linear(hidden_dim, output_dim))
        self.drop = nn.Dropout(0.5)
        if pooling == 'sum'
            self.pool = SumPooling() # change to mean readout (AvgPooling) on social network datasets
        elif pooling == 'avg':
            self.pool = AvgPooling()
        else:
            self.pool = MaxPooling()

    def forward(self, g, h):
        # list of hidden representation at each layer (including the input layer)
        hidden_rep = [h]
        for i, layer in enumerate(self.ginlayers):
            h = layer(g, h)
            h = self.batch_norms[i](h)
            h = F.relu(h)
            hidden_rep.append(h)
        score_over_layer = 0
        # perform graph sum pooling over all nodes in each layer
        for i, h in enumerate(hidden_rep):
            pooled_h = self.pool(g, h)
            score_over_layer += self.drop(self.linear_prediction[i](pooled_h))
        return score_over_layer
    
def split_fold10(labels, fold_idx=0):
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    idx_list = []
    for idx in skf.split(np.zeros(len(labels)), labels):
        idx_list.append(idx)
    train_idx, valid_idx = idx_list[fold_idx]
    return train_idx, valid_idx

def evaluate(dataloader, device, model):
    model.eval()
    total = 0
    total_correct = 0
    for batch, (batched_graph, labels) in enumerate(tqdm(train_loader)):
        batched_graph = batched_graph.to(device)
        labels = labels.to(device)
        feat = batched_graph.ndata.pop('x')
        total += len(labels)
        logits = model(batched_graph, feat.view(len(feat), -1))
        _, predicted = torch.max(logits, 1)
        total_correct += (predicted == labels).sum().item()
    acc = 1.0 * total_correct / total
    return acc

def train(train_loader, val_loader, test_loader, device, model):
    # loss function, optimizer and scheduler
    loss_fcn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)
    
    # training loop    
    for epoch in range(350):
        model.train()
        total_loss = 0
        for batch, (batched_graph, labels) in enumerate(tqdm(train_loader)):
            batched_graph = batched_graph.to(device)
            #print(batch, labels, type(labels))
            labels = labels.to(device)
            # print(labels)
            feat = batched_graph.ndata.pop('x')
            # print(feat.view(50,-1).shape)
            logits = model(batched_graph, feat.view(len(feat), -1))
            # print(logits)
            # print(logits.shape, labels.shape)
            loss = loss_fcn(logits, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        scheduler.step()
        train_acc = evaluate(train_loader, device, model)
        valid_acc = evaluate(val_loader, device, model)
        test_acc = evaluate(test_loader, device, model)
        print("Epoch {:05d} | Loss {:.4f} | Train Acc. {:.4f} | Validation Acc. {:.4f}| Test Acc. {:.4f} "
              . format(epoch, total_loss / (batch + 1), train_acc, valid_acc, test_acc))

if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--dataset', type=str, default="MUTAG",
#                         choices=['MUTAG', 'PTC', 'NCI1', 'PROTEINS'],
#                         help='name of dataset (default: MUTAG)')
#     args = parser.parse_args()
    print(f'Training with DGL built-in GINConv module with a fixed epsilon = 0')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # load and split dataset
    # dataset = GINDataset(args.dataset, self_loop=True, degree_as_nlabel=False) # add self_loop and disable one-hot encoding for input features
    dataset = HomoStrucBackdoorDataset()
    dataset.load()
    labels  = [y for y in dataset.iter_y()]
    
    # train_idx, val_idx = split_fold10(labels)
    # print(train_idx, val_idx)
    
    # create dataloader
    train_loader = GraphDataLoader(dataset, batch_size=4, pin_memory=torch.cuda.is_available())
    val_dataset = HomoStrucBackdoorDataset(mode='valid')
    val_loader = GraphDataLoader(val_dataset, batch_size=4, pin_memory=torch.cuda.is_available())
    test_dataset = HomoStrucBackdoorDataset(mode='test')
    test_loader = GraphDataLoader(test_dataset, batch_size=4, pin_memory=torch.cuda.is_available())
    # create GIN model
    in_size = 512 * 513
    gin_dataset = GINDataset('MUTAG', self_loop=True, degree_as_nlabel=False) # add self_loop and disable one-hot encoding for input features
    # print(gin_dataset.dim_nfeats)
    out_size = 2
    model = GIN(in_size, 16, out_size).to(device)

    # model training/validating
    print('Training Procedure...')
    train(train_loader, val_loader, test_loader, device, model)

Training with DGL built-in GINConv module with a fixed epsilon = 0
Training Procedure...


100%|██████████| 1024/1024 [00:46<00:00, 22.08it/s]


Epoch 00000 | Loss 2.6127 | Train Acc. 0.9248 | Validation Acc. 0.9248| Test Acc. 0.9248 


100%|██████████| 1024/1024 [00:46<00:00, 22.13it/s]


Epoch 00001 | Loss 0.9778 | Train Acc. 0.9651 | Validation Acc. 0.9651| Test Acc. 0.9651 


...

Epoch 00040 | Loss 0.0421 | Train Acc. 1.0000 | Validation Acc. 1.0000| Test Acc. 1.0000 


100%|██████████| 1024/1024 [00:46<00:00, 21.93it/s]


Epoch 00041 | Loss 0.0404 | Train Acc. 1.0000 | Validation Acc. 1.0000| Test Acc. 1.0000 


 68%|██████▊   | 699/1024 [00:39<00:18, 17.61it/s]


KeyboardInterrupt: 

In [53]:
train_loader = GraphDataLoader(dataset, batch_size=4, pin_memory=torch.cuda.is_available())
for batch, (batched_graph, labels) in enumerate(train_loader):
    print(batch, )
    print((batched_graph, labels))
    break

0
(Graph(num_nodes=200, num_edges=2180,
      ndata_schemes={'x': Scheme(shape=(512, 513), dtype=torch.float32)}
      edata_schemes={}), tensor([0, 0, 1, 0]))


TypeError: object of type 'DGLHeteroGraph' has no len()